In [1]:
import numpy as np 
import pandas as pd

### Dependencies

In [3]:
import pandas as pd
import tensorflow.keras as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
num_columns = 785

# Create column names
columns = ['label'] + list(range(1, num_columns))

# Read the training and testing datasets
ds_train = pd.read_csv("emnist-balanced-train.csv", header=None)
ds_test = pd.read_csv("emnist-balanced-test.csv", header=None)

# Assign column names to the DataFrames
ds_train.columns = columns
ds_test.columns = columns

# Reset index
ds_train = ds_train.reset_index(drop=True)
ds_test = ds_test.reset_index(drop=True)

# Extract features and labels
x_train = ds_train.drop(['label'], axis=1)
y_train = ds_train['label']
x_test = ds_test.drop(['label'], axis=1)
y_test = ds_test['label']

# Display the first few rows of the DataFrame
print(ds_train.head())
print(ds_test.head())

###  62 classes , mapped to actual values

In [ ]:
alpha_num_to_char = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'a', 11: 'b', 12: 'c', 13: 'd', 14: 'e', 15: 'f', 16: 'g', 17: 'h', 18: 'i', 19: 'j',
    20: 'k', 21: 'l', 22: 'm', 23: 'n', 24: 'o', 25: 'p', 26: 'q', 27: 'r', 28: 's', 29: 't',
    30: 'u', 31: 'v', 32: 'w', 33: 'x', 34: 'y', 35: 'z',
    36: 'A', 37: 'B', 38: 'C', 39: 'D', 40: 'E', 41: 'F', 42: 'G', 43: 'H', 44: 'I', 45: 'J',
    46: 'K', 47: 'L', 48: 'M', 49: 'N', 50: 'O', 51: 'P', 52: 'Q', 53: 'R', 54: 'S', 55: 'T',
    56: 'U', 57: 'V', 58: 'W', 59: 'X', 60: 'Y', 61: 'Z'
}

In [ ]:
def show_example(n) :
    label = alpha_num_to_char[ds_train.iloc[n,0]]  # the example's label
    plt.title(label)
    example = np.array(x_train.iloc[n]) # extract the 28 *28 array that contains the pixels
    example_image = example.reshape(28 , 28 ) # rearrange the array to be a 28 by 28 matrix ,that represents that actual image
    plt.imshow(example_image , cmap ='gray')
    plt.axis('off')
  
    plt.show()

In [ ]:
def show_example_and_predicted_val(n,predicted_n) :

    true_val = alpha_num_to_char[ds_train.iloc[n,0]]  # the example's label
    predicted_val = alpha_num_to_char[predicted_n]  # the example's label
    
    plt.title("Actual val : "+ true_val+"\n Predicted :" + predicted_val)
    example = np.array(x_train.iloc[n]) # extract the 28 *28 array that contains the pixels
    example_image = example.reshape(28 , 28 ) # rearrange the array to be a 28 by 28 matrix ,that represents that actual image
    plt.imshow(example_image , cmap ='gray')
    plt.axis('off')
  
    plt.show()

In [ ]:
show_example(3)

### Preprocessing using a pipeline

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

class Reshaper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_reshaped = X.reshape(-1, 28, 28, 1)
        return X_reshaped

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: StandardScaler
    ('reshaper', Reshaper())  # Step 2: Custom Reshaper
])


# Fit and transform the training data

x_train_processed = pipeline.fit_transform(x_train)
y_train_processed = np.eye(62)[y_train.astype(int)]

x_test_processed = pipeline.transform(x_test)

y_test_processed = np.eye(62)[y_test.astype(int)]

In [ ]:
print('input shape',x_train_processed.shape)
print('output shape',y_train.shape)
# y_train = np.eye(62)[y_train.astype(int)]
print('output shape',y_train_processed.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout

#  multiple model architectures
models = []
input_shape=(28,28,1)
# Model 1
model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(62, activation='softmax'))
models.append(model1)

# Model 2
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model2.add(Dropout(0.25)) 
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.4))
model2.add(Dense(62, activation='softmax'))
# Add layers for Model 2
models.append(model2)

# Model 3
model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model3.add(Dropout(0.25)) 
model3.add(MaxPooling2D((2, 2)))
model3.add(Flatten())
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.4))
model3.add(Dense(128, activation='relu'))
model3.add(Dense(62, activation='softmax'))
models.append(model3)

# Train each model
for i, model in enumerate(models):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train_processed, y_train_processed, epochs=5, batch_size=32)

# Evaluate each model
accuracies = []
for i, model in enumerate(models):
    loss, accuracy = model.evaluate(x_test_processed, y_test_processed)
    accuracies.append(accuracy)
    print(f"Model {i+1} Accuracy: {accuracy}")

# Select the best model based on accuracy
best_model_index = accuracies.index(max(accuracies))
best_model = models[best_model_index]
print(f"Best Model: Model {best_model_index+1}")

In [ ]:
# some example from the data set
x=65765
example0=np.argmax(best_model.predict(x_train_processed[x].reshape(1,28,28,1)).flatten())
alpha_num_to_char[example0]
show_example_and_predicted_val(x,example0)

In [ ]:
import pickle

In [ ]:
# pickle.dump(df,open('df.pkl','wb'))
# pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
best_model.save('best_model.h5')

In [ ]:
from keras.models import load_model

# Load the saved model
loaded_model = load_model('best_model.h5')

# Now you can use the loaded_model for inference or further training


In [ ]:
import numpy as np
# some example from the data set
x=68785
example0=np.argmax(loaded_model.predict(x_train_processed[x].reshape(1,28,28,1)).flatten())
alpha_num_to_char[example0]
show_example_and_predicted_val(x,example0)